# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of the natural language (could be sequences or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Quadro P2000 (UUID: GPU-45eb6a28-8b9a-6cc9-5299-32453fd1cd04)


## Get the helper functions 

In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we are going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

In [4]:
# Unzip data
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way would be to use python.

Another way to do this is pandas

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Visualizing a sample
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [10]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [17]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Terrorists attack police post; 2 SPOs injured http://t.co/lXMdiseUCn #YUG

---

Target: 0 (not real disaster)
Text:
@_itzSteven @xdojjjj @whopper_jr_760 huh?? me you and leo started that last year and ever since people blaze it in the back??

---

Target: 1 (real disaster)
Text:
:StarMade: :Stardate 3: :Planetary Annihilation:: http://t.co/I2hHvIUmTm via @YouTube

---

Target: 0 (not real disaster)
Text:
Dan Hughes was taken off on a stretcher after Danielle Robinson collided @OU_WBBall hope they are both ok he called SOME  games over years

---

Target: 0 (not real disaster)
Text:
@AlexJacobsonPFS All Andre and Gore have to do is not fall off the cliff and we're elite on that side of the ball.

---



### Split data into training and validation sets

In [21]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation
                                                                            random_state=42)


In [22]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [23]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

In NLP, there are two main concepts for turning text into numbers:

* Tokenization - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:

* Embeddings - An embedding is a representation of natural language which can be learned

### Text vectorization (tokenization)

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length=None) # pads sequence to the largest sequence
                                    # pad_to_max_tokens=True)

In [33]:
# Length of train setences
print(train_sentences[0].split())
len(train_sentences[0].split())

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']


7

In [34]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [35]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocab
max_length = 15 # the length of our sequences (i.e. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [36]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [37]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [41]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])      

Original text:
 #3Novices : Renison mine sees seismic event http://t.co/2i4EOGGO5j A small earthquake at Tasmania's Renison tin project has created a temÛ_      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 4759, 1635, 1768,  641, 1414,    1,    3, 1111,  290,   17,
        4467, 4759, 4407, 2016]], dtype=int64)>

In [42]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our vocab
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

To make our embedding, we're going to use TF embedding layer.

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `ouput_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [43]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # input shape
                             output_dim=128, # output shape
                             input_length=max_length) # how long is each input

In [47]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n {random_sentence} \
        \n\n Embedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
 Rainstorm Update: Further North or Further South? http://t.co/50vdQ7A1M5 http://t.co/QH6oXfT9Ir         

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03502663,  0.03921941,  0.0383674 , ..., -0.02635555,
          0.03947334, -0.03340974],
        [-0.01292474, -0.00494833, -0.04655982, ...,  0.01758016,
         -0.03785324,  0.03631197],
        [ 0.01070427,  0.01867845,  0.02958337, ...,  0.03754005,
         -0.04198488,  0.00847014],
        ...,
        [-0.01541116,  0.00267478,  0.02568301, ...,  0.02798783,
         -0.03544172,  0.02741717],
        [-0.01541116,  0.00267478,  0.02568301, ...,  0.02798783,
         -0.03544172,  0.02741717],
        [-0.01541116,  0.00267478,  0.02568301, ...,  0.02798783,
         -0.03544172,  0.02741717]]], dtype=float32)>

In [50]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03502663,  0.03921941,  0.0383674 ,  0.00750684, -0.04518265,
        -0.01214756, -0.00297828,  0.00466927, -0.01365813, -0.0374611 ,
        -0.01708464, -0.01776303,  0.01560349, -0.01901287,  0.00594134,
         0.01593817,  0.00017445, -0.0216028 ,  0.04213203, -0.00975668,
         0.03518666, -0.00732756, -0.02087513, -0.03335022,  0.01269842,
         0.01470233, -0.00121951,  0.03410668, -0.00089784,  0.04774914,
         0.0164078 , -0.04479611, -0.0450394 ,  0.04537685,  0.0050956 ,
         0.02569455, -0.01367582,  0.02685616,  0.04452941,  0.04921999,
         0.04004481,  0.02332344,  0.02186411,  0.03473711,  0.04160433,
         0.01219344, -0.01888226,  0.00190976, -0.01772344, -0.02165477,
        -0.02468081, -0.04173925, -0.04806926, -0.04749493, -0.04460694,
         0.04915437,  0.03054347,  0.0217176 ,  0.0075479 ,  0.04508701,
        -0.00158276,  0.04349719,  0.008368  , -0.04198517,  0.03071276,
  

## Modelling a text dataset (running a series of experiments)

Now we have got a way to turn our text sequences into numbers, it is time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM (RNN)
* Model 3: GRU (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D CNN
* Model 6: TF Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as Model 6 with 10% of training data

How are we going to approach all of these?
1. Create a model
2. Build a model
3. Fit a model
4. Evaluate a model

### Model 0: Getting a baseline

> It is common practice to use non-DL algorithms as a baseline because of their speed. Later, we can employ deep learning and see how it improves the result

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), # convert words to numbers
    ("clf",MultinomialNB()), # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [53]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [54]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predicitons with difference metrics ever time, however, this can be fixed with a function

Let's create a function to compare our model's predictions to its truth labels using:
* Accuracy
* Precision
* Recall
* F1-Score

In [55]:
# Function to evaluate metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    """

    # Calculate accuracy
    model_accuracy = accuracy_score(y_true,y_pred) * 100
    # Calculate precision, recall, f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true,y_pred,average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}

    return model_results                         

In [56]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results                              

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [57]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_log_nlp"

In [63]:
# Build a model with the Function API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1,activation="sigmoid")(x) # Create the output layer
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [64]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [65]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_log_nlp/model_1_dense/20221229-104303
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.6102 - accuracy: 0.6901 - val_loss: 0.5334 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4422 - accuracy: 0.8181 - val_loss: 0.4707 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3472 - accuracy: 0.8612 - val_loss: 0.4562 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2851 - accuracy: 0.8913 - val_loss: 0.4640 - val_accuracy: 0.7953
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2387 - accuracy: 0.9133 - val_loss: 0.4751 - val_accuracy: 0.7887


In [66]:
# Check the results 
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4751 - accuracy: 0.7887


[0.47509241104125977, 0.7887139320373535]

In [69]:
# Make predicitons
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [70]:
# Look at a single prediction
model_1_pred_probs[0]

array([0.35178968], dtype=float32)

In [71]:
# Look at the first 10 predicitons
model_1_pred_probs[:10]

array([[0.35178968],
       [0.7886459 ],
       [0.9971089 ],
       [0.11678676],
       [0.10056055],
       [0.922948  ],
       [0.90426356],
       [0.9938454 ],
       [0.9574441 ],
       [0.26181763]], dtype=float32)

In [73]:
# Convert model prediction probabilites to label format
model_1_pred_probs = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred_probs[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [78]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_pred_probs)
model_1_results, baseline_results                         

({'accuracy': 78.87139107611549,
  'precision': 0.793474487062673,
  'recall': 0.7887139107611548,
  'f1': 0.7858143716287432},
 {'accuracy': 79.26509186351706,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706,
  'f1': 0.7862189758049549})

In [76]:
# Looks like our baseline is out performing our first deep learning model
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])